In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from PIL import Image

input_folder = 'NLM Filters/test/severe_dysplastic_filtered_nlm'
output_folder = 'Augmentation/test/severe_dysplastic_filtered_nlm_aug'

# Create an ImageDataGenerator and specify the augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=45,      # Rotate the image between -45 to 45 degrees
    width_shift_range=0.2,  # Shift the width of the image by up to 20%
    height_shift_range=0.2, # Shift the height of the image by up to 20%
    shear_range=0.2,        # Apply shear transformation with a shear factor of 0.2
    zoom_range=0.2,         # Zoom the image by up to 20%
    horizontal_flip=True,   # Flip the image horizontally
    vertical_flip=True     # Flip the image vertically
    # Add more augmentation parameters here
)

# Generate augmented images and save them to the output folder
for filename in os.listdir(input_folder):
    image_path = os.path.join(input_folder, filename)

    # Load the image using Keras' load_img function
    image = load_img(image_path)

    # Convert the image to a NumPy array
    image_array = img_to_array(image)

    # Reshape the image array to match the expected input shape of the generator
    image_array = image_array.reshape((1,) + image_array.shape)

    # Generate augmented images using the datagen.flow() method
    augmented_images = datagen.flow(
        image_array,
        batch_size=1,
        save_to_dir=output_folder,  # Save the augmented images to the output folder
        save_prefix='augmented',    # Prefix to use for the saved images
        save_format='png'           # Save images in PNG format
    )

    # Generate and save the augmented images
    num_augmented_images = 5  # Specify the number of augmented images to generate
    for i, augmented_image in enumerate(augmented_images):
        if i >= num_augmented_images:
            break

        # Convert the augmented image to PIL format
        augmented_image_pil = Image.fromarray(augmented_image[0].astype('uint8'))

        # Save the augmented image as PNG
        save_filename = f'{filename.split(".")[0]}_{i}.png'
        save_path = os.path.join(output_folder, save_filename)
        augmented_image_pil.save(save_path)
print("Augmented images done ")


FileNotFoundError: [Errno 2] No such file or directory: 'Augmentation/test/severe_dysplastic_filtered_nlm_aug\\augmented_0_1157.png'

Found 6951 images belonging to 7 classes.
Found 3001 images belonging to 7 classes.
Epoch 1/5
434/434 [==============================] - 652s 1s/step - loss: 1.9109 - accuracy: 0.2016 - val_loss: 1.9038 - val_accuracy: 0.1641
Epoch 2/5
434/434 [==============================] - 683s 2s/step - loss: 1.9114 - accuracy: 0.1934 - val_loss: 1.9117 - val_accuracy: 0.1975
Epoch 3/5
434/434 [==============================] - 666s 2s/step - loss: 1.9115 - accuracy: 0.1994 - val_loss: 1.8935 - val_accuracy: 0.2129
Epoch 4/5
434/434 [==============================] - 641s 1s/step - loss: 1.9045 - accuracy: 0.2072 - val_loss: 1.9056 - val_accuracy: 0.2129
Epoch 5/5
188/188 [==============================] - 177s 940ms/step


Number of images in the training set: 6951
Number of images in the test set: 3001
Number of features extracted per image: 1280


In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Set the paths to the dataset
train_data_dir = 'NLM Augmentation/train'
test_data_dir = 'NLM Augmentation/test'

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 16

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1. / 255)

# Load the pretrained model for feature extraction
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the base model's layers
base_model.trainable = False

# Define the number of classes based on your dataset
num_classes = 7

# Transformer-based feature extraction
inputs = Input(shape=(img_width, img_height, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to see the layer names
model.summary()

# Set up data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Specify the number of training and validation steps
train_steps = train_generator.samples // batch_size
val_steps = test_generator.samples // batch_size

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=5,
    validation_data=test_generator,
    validation_steps=val_steps,
    callbacks=[EarlyStopping(patience=3)]
)

# Get the names of all the columns from the GlobalAveragePooling2D layer
column_names = model.layers[-2].get_config()['name'] + '_0'

print("Column names from feature extraction:")
print(column_names)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 7)                 8967      
                                                                 
Total params: 4,058,538
Trainable params: 8,967
Non-trainable params: 4,049,571
_________________________________________________________________
Found 6951 images belonging to 7 classes.
Found 3001 images belonging to 7 classes.
Epoch 1/5
434/434 [==========

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Set the paths to the dataset
train_data_dir = 'NLM Augmentation/train'
test_data_dir = 'NLM Augmentation/test'

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 16

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1. / 255)

# Load the pretrained model for feature extraction
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the base model's layers
base_model.trainable = False

# Define the number of classes based on your dataset
num_classes = 7

# Transformer-based feature extraction
inputs = Input(shape=(img_width, img_height, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Specify the number of training and validation steps
train_steps = train_generator.samples // batch_size
val_steps = test_generator.samples // batch_size

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=5,
    validation_data=test_generator,
    validation_steps=val_steps,
    callbacks=[EarlyStopping(patience=3)]
)

# Create a new model with the feature outputs as the output layer
feature_extraction_model = Model(inputs=model.input, outputs=x)

# Get the names of the features (layers in the feature_extraction_model)
feature_names = [layer.name for layer in feature_extraction_model.layers]

print("Names of the features extracted:")
print(feature_names)


Found 6951 images belonging to 7 classes.
Found 3001 images belonging to 7 classes.
Epoch 1/5
434/434 [==============================] - 483s 1s/step - loss: 1.9071 - accuracy: 0.1902 - val_loss: 1.8941 - val_accuracy: 0.1574
Epoch 2/5
434/434 [==============================] - 463s 1s/step - loss: 1.9064 - accuracy: 0.1935 - val_loss: 1.9014 - val_accuracy: 0.2129
Epoch 3/5
434/434 [==============================] - 469s 1s/step - loss: 1.9031 - accuracy: 0.1968 - val_loss: 1.9095 - val_accuracy: 0.2129
Epoch 4/5
434/434 [==============================] - 484s 1s/step - loss: 1.9015 - accuracy: 0.1990 - val_loss: 1.8858 - val_accuracy: 0.2129
Epoch 5/5
434/434 [==============================] - 474s 1s/step - loss: 1.9075 - accuracy: 0.2016 - val_loss: 1.9425 - val_accuracy: 0.1641
Names of the features extracted:
['input_4', 'efficientnetb0', 'global_average_pooling2d_1']
